In [2]:
# get all imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, log_loss # if you're on mac using miniconda miniforge3 install miniforge::compilers to fix issue with this module
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import numpy as np
import os

In [8]:
# Define data transforms from the Ai_Competition github
data_transforms = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop(512),        # Randomly crop the image to 512x512
            transforms.RandomHorizontalFlip(),        # Randomly flip the image horizontally
            transforms.ToTensor(),                    # Convert the image to a tensor
            transforms.Normalize([0.485, 0.456, 0.406], # Normalize with standard values
                                 [0.229, 0.224, 0.225]) # Mean and Std for RGB channels
        ]),
        'val': transforms.Compose([
            transforms.Resize(512),                    # Resize the image to 512x512
            transforms.CenterCrop(512),                # Crop from the center
            transforms.ToTensor(),                     # Convert the image to a tensor
            transforms.Normalize([0.485, 0.456, 0.406], # Normalize with standard values
                                 [0.229, 0.224, 0.225]) # Mean and Std for RGB channels
        ]),
    }

In [35]:
data_dir = "AI_Competition"
batch_size=32
print(os.getcwd())
image_datasets = {x: datasets.ImageFolder(data_dir,
    data_transforms[x])
    for x in ['train', 'val']}

dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size,
    shuffle=True, num_workers=4)
    for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

/Users/kevincrothers/Documents/GitHub/Regression-Renegades-MSU-Ai-Competiition


FileNotFoundError: Found no valid file for the classes Butler Hall, Lee Hall, McCain Hall, Student Union, Walker Hall. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp